Purpose of the notebook:

* Check how many NAN values exist in the array
* Visual distribution of variables Testing the predictive power of each variable
* Checking relationships between independent dinners (pairs) and the type of relationship:

Category -> Category

Category -> Quantitative

Quantitative -> Quantitative



Expected insights:
*  Is there enough data to solve the questions
*  Do engineer more features
*  If necessary which features delete first
  


In [ ]:
import torch
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import time 
from scipy import stats

from collections import Counter
from clean import *
from control import *
from  graphs import *

In [ ]:
Graphs.v()
s = time.time()

In [ ]:
data = pd.read_csv("/kaggle/input/characteristics-corona-patients/Characteristics_Corona_patients version 5 06122020.csv",
                   parse_dates=['confirmed_date', 'deceased_date',"released_date","return_date", "date_onset_symptoms"])

In [ ]:
data.shape

In [ ]:
data.columns

# nan distribution

In [ ]:
plt.figure(figsize = (16,8))

ax = sns.heatmap(data.isnull(), cbar=False)

In [ ]:
for j in ['age', 'age_band', 'background_diseases', 'background_diseases_binary',
       'city', 'confirmed_date', 'country', 'date_onset_symptoms',
       'deceased_date',  'infected_by', 'origin', 'region',
       'released_date', 'return_date', 'severity_illness', 'sex', 'smoking',
       'symptoms', 'treatment', 'confirmed_date_D', 'deceased_date_D',
       'released_date_D', 'return_date_D', 'date_onset_symptoms_D',
  
       'date_onset_symptoms_until_confirmed_date',
       'confirmed_date_until_released_date',
       'confirmed_date_until_deceased_date',
       'severity_illness_infectious_person', 'len_people_infected_by_patient']:
    print(j, str(len(data.index[data[j].notnull()])))


# The distribution of properties in the dataset

In [ ]:

fig, ax = plt.subplots(9, 2, figsize=(15,45))
palette_severity_illness = {"asymptomatic": "#C77DE7", "good":"yellowgreen", "critical":"red", "deceased": "black", "cured": "#0BEFFF"}

# 0
# severity_illness
Graphs.basic_countplot(data.severity_illness, 'severity_illness', ax, (0,0), "y", palette_severity_illness)

# severity_illness_infectious_person
Graphs.basic_countplot(data.severity_illness_infectious_person, 'severity_illness_infectious_person', 
                ax, (0,1), "y", palette_severity_illness)


# line 1
# sex
Graphs.basic_countplot(data.sex, 'sex',  ax, (1,0), "y", None)

# treatment
Graphs.basic_countplot(data.treatment, 'treatment',  ax, (1,1), "y", None)


# line 2
# age_band
Graphs.hist_graf(data.age_band, 13, "age band", ax,(2,1), "y")

# age
Graphs.hist_graf(data.age, 120, "age", ax ,(2,0), "y")


# line 3
# return_date
Graphs.time_plot(data.return_date, "return_date", ax ,(3,0), "y", "blue")
Graphs.hist_graf(data.len_people_infected_by_patient, 100, "len_people_infected_by_patient", ax,(3,1), "y")


# line 4
# date_onset_symptoms
Graphs.time_plot(data.date_onset_symptoms, "date_onset_symptoms", ax ,(4,0), "y", "blue")

# confirmed_date
Graphs.time_plot(data.confirmed_date, "confirmed_date", ax ,(4,1), "y", "blue")


# line 5
# deceased_date
Graphs.time_plot(data.deceased_date, "deceased_date", ax ,(5,0), "y", "red")

# released_date
Graphs.time_plot(data.released_date, "cured_date", ax ,(5,1), "y", "blue")


#  line 6
# background_diseases_binary
Graphs.basic_countplot(data.background_diseases_binary, 'background_diseases_binary', 
                ax, (6,0), "y", None)

# smoking
Graphs.basic_countplot(data.smoking, 'smoking', ax, (6,1), "y", None)


#  line 7
# return_date_until_date_onset_symptoms
# Graphs.hist_graf(data.return_date_until_date_onset_symptoms, 120,
#           "return_date_until_date_onset_symptoms", ax, (7,0), "y")

# date_onset_symptoms_until_confirmed_date
Graphs.hist_graf(data.date_onset_symptoms_until_confirmed_date, 100,
          "date onset symptoms until confirmed date", ax,(7,1), "y")

#  line 8
# confirmed_date_until_released_date
Graphs.hist_graf(data.confirmed_date_until_released_date, 100,
          "confirmed_date_until_released_date", ax,(8,0), "y")

# confirmed_date_until_deceased_date
Graphs.hist_graf(data.confirmed_date_until_deceased_date, 100,
          "confirmed_date_until_deceased_date", ax,(8,1), "y")

fig.tight_layout()
plt.show()

many j

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(15,45))
Graphs.h_bar_with_limit(data, 'symptoms', 40 , "symptoms", ax, 0 , "x", "blue")

Graphs.h_bar_with_limit(data, 'background_diseases', 300 , 'background_diseases', ax, 1 , "x", "blue")

Graphs.h_bar_with_limit(data, 'country', 4000 , 'country', ax, 2 ,  "x", "blue")

plt.show()

# Correlation between 2 variables - visual

## quantitative -> quantitative

In [ ]:
df_random_sample = data.sample(frac=0.005, replace=True, random_state=1)

In [ ]:
df_random_sample.shape

In [ ]:
for x in ["age",  "date_onset_symptoms_D"]: 
    print(x)
    for y in ["date_onset_symptoms_until_confirmed_date",
              "confirmed_date_until_deceased_date", 
              "confirmed_date_until_released_date"]:
        sns.regplot(x, y, marker="+", data=df_random_sample)
        plt.show()

Insights:

Older people die faster from covid19

As time went on people recovered faster - probably due to a change in the definition of "healed person"


In [ ]:
df2 = data[data.len_people_infected_by_patient.notnull()]

In [ ]:
for x in ["age", "date_onset_symptoms_D"]:
    sns.scatterplot(x, "len_people_infected_by_patient", marker="+", data=df2)
    plt.show()

Insights:

There is information on how many people infected the patient only at the beginning of the event


There is a slight increase in the number of people infected by the patient according to his age (the older he is, the more people are infected)


possible reasons

* Older patients report contacts more accurately
* Older people see people their age so those who are infected tend to develop more severe symptoms (thus "identifying" as patients)

## category -> category

### Chi-Square Table Test

In [ ]:
def make_1value_matrix(cols, value):
    neg1_matrix = pd.DataFrame(index=cols)
    for j in cols:
        neg1_matrix[j]= value
    return neg1_matrix
    
def make_chi2_corr(data, cols, basic_value):
    chi2_corr= make_1value_matrix(cols, basic_value)
    for j in cols:
        for r in cols:
            if len(data[data[r].notnull() & data[j].notnull()]) >200:
                crosstab = pd.crosstab(data[j],  data[r])
                chi2,  p, dof, exp =  stats.chi2_contingency(crosstab)
                
                if p < 0.0001:
                    chi2_corr.loc[r, j] = 1
                else:
                    chi2_corr.loc[r, j] = 0
    return chi2_corr

In [ ]:
cols = ["origin" ,
"severity_illness",
'sex',
"country",
"region",
"city",
"background_diseases_binary",
"treatment",  
"smoking",
"severity_illness_infectious_person"]

chi2_corr = make_chi2_corr(data, cols, -1)

In [ ]:
sns.heatmap(chi2_corr, annot=True)

1 = The two variables are interdependent
( p-value <= 0.01)

0 = Two variables independent of each other
( p-value > 0.01)

-1 = There is not enough information to decide
(Less than 200 rows containing data in both columns)


# howmach databases show the correlation between the variables?
- Any database that shows a correlation is considered as 1
- If 0 or there were not enough rows or no buffer separately showed the correlation

In [ ]:
matrix = make_1value_matrix(cols,0)
for origin in ["world","colombia","mexico","canada","toronto","kerla","korea",
               "hong_kong","singapore","france","india_data",'indonesia',"philippines",
               "japan","vietnam","india_wiki","new_zealand","guatemala","tunisia"]:
    x  = data[data.origin == origin]
    chi2_corr_origin = make_chi2_corr(x, cols, 0)
    matrix += chi2_corr_origin

In [ ]:
sns.heatmap(matrix, annot=True)

## category -> quantitative

In [ ]:
def hist_by_categories(data, q_col, c_col, title ,  ax , axs):
    value_count_dict = dict(data[c_col].value_counts())
    for k in value_count_dict:
        sns.kdeplot(data.loc[(data[c_col]==k),q_col], shade=True, Label=k, ax=ax[axs]) 
        
    ax[axs].set_xlabel(q_col) 
    ax[axs].set_ylabel('probability density') 
    ax[axs].set_title(title,fontsize=20)


In [ ]:
q = ['age', 'confirmed_date_D',
'len_people_infected_by_patient',
'date_onset_symptoms_until_confirmed_date',
'confirmed_date_until_released_date',
'confirmed_date_until_deceased_date']



c = [ 'background_diseases_binary',
'origin',  'severity_illness', 'sex', 'smoking',
# 'symptoms', 'background_diseases',
     'treatment','severity_illness_infectious_person']
fig, ax = plt.subplots(len(c) * len(q)+1, figsize=(9, 300)) 
counte = 0  
for c_index in range(len(c)):

    for q_index in range(len(q)):
        title = q[q_index]+ " VS " + c[c_index]
        hist_by_categories(data, q[q_index], c[c_index], title ,  ax , (counte ))
        counte +=1 


## correlation

people infected by the patient - age

confirmed_date_until_deceased_date - age

severity_illness - age



## bios
Most of the information in the dataset is missing

There is information on how many people infected the patient only at the beginning of the event

As time went on people recovered faster - probably due to a change in the definition of "healed person"

There is information on infection only in patients with background diseases

In [ ]:
data.columns